In [1]:
import numpy as np
import sympy as sym
import numba
import pydae.build as db

\begin{eqnarray}
\dot \delta &=& \Omega_b \left(\omega - \omega_s\right)\\
\dot \omega &=& 1/(2 H) \left(p_m - p_e - D (\omega - \omega_s) \right)\\
\end{eqnarray}

$$ \sf
\Omega_{b} \left(\sf \omega - \omega_{s}\right)
$$

## System definition 

In [2]:
params = {'S_b':500.0,
          'X_d':1.81,'X1d':0.3,'T1d0':8.0,   # synnchronous machine d-axis parameters
          'X_q':1.76,'X1q':0.65,'T1q0':1.0,  # synnchronous machine q-axis parameters
          'R_a':0.003,'X_l': 0.05, 
          'H':3.5,'D':1.0,
          'Omega_b':2*np.pi*50,'omega_s':1.0,
          'v_0':1.0,'theta_0':0.0,
          'K_a':100, 'T_r':0.1, 'v_pss':0.0,
          'T_c':2.0,'T_b':10.0,'Kpgov':10.0,'Kigov':2.0,'Droop':0.05,'Kimw':0.0,'S_b_g':500.0}
                            
u_ini_dict = {'P_t':0.8, 'Q_t':0.2}  # for the initialization problem
u_run_dict = {'p_m':0.8,'v_ref':1.0}  # for the running problem (here initialization and running problem are the same)


x_list_syn = ['delta','omega','e1q','e1d','v_c']    # [inductor current, PI integrator]
y_ini_list = ['i_d','i_q','v_1','theta_1','p_m','v_ref','v_f'] # for the initialization problem
y_run_list = ['i_d','i_q','v_1','theta_1','P_t','Q_t',  'v_f'] # for the running problem (here initialization and running problem are the same)

x_list_gov = ['xi_gov','xi_mw','x_gov','p_m_meas']    # [inductor current, PI integrator]


sys_vars = {'params':params,
            'u_list':u_run_dict,
            'x_list':x_list_syn + x_list_gov,
            'y_list':y_run_list}

exec(db.sym_gen_str())  # exec to generate the required symbolic varables and constants

In [7]:
v_d = v_1*sin(delta - theta_1) 
v_q = v_1*cos(delta - theta_1) 

p_e = i_d*(v_d + R_a*i_d) + i_q*(v_q + R_a*i_q) 


ddelta = Omega_b*(omega - omega_s)
domega = 1/(2*H)*(p_m - p_e - D*(omega - omega_s))
de1q = 1/T1d0*(-e1q - (X_d - X1d)*i_d + v_f)
de1d = 1/T1q0*(-e1d + (X_q - X1q)*i_q)
dv_c =   (v_1 - v_c)/T_r

g_1 = v_q + R_a*i_q + X1d*i_d - e1q
g_2 = v_d + R_a*i_d - X1q*i_q - e1d
g_3 = P_t - (v_1*v_0*sin(theta_1 - theta_0))/X_l
g_4 = Q_t + (v_1*v_0*cos(theta_1 - theta_0))/X_l - v_1**2/X_l
g_5 = i_d*v_d + i_q*v_q - P_t
g_6 = i_d*v_q - i_q*v_d - Q_t
g_7 = K_a*(v_ref - v_c + v_pss) - v_f 


sys = {'name':'smib_milano_ex8p1_4ord_avr',
       'params_dict':params,
       'f_list':[ddelta,domega,de1q,de1d,dv_c],
       'g_list':[g_1,g_2,g_3,g_4,g_5,g_6,g_7],
       'x_list':x_list,
       'y_ini_list':y_ini_list,
       'y_run_list':y_run_list,
       'u_run_dict':u_run_dict,
       'u_ini_dict':u_ini_dict,
       'h_dict':{'p_m':p_m}}

sys = db.system(sys)
db.sys2num(sys)
                

IndexError: list index out of range

In [6]:


omega_droop = Droop*p_m_meas
p_gov= Kpgov*(omega_ref - omega - omega_droop + Kimw*xi_mw) + Kigov*xi_gov
p_m = x_gov + T_c_1/T_b_1*(p_gov - x_gov)
f1 =  1.0 - omega_1 - omega_droop_1 + Kimw_1*xi_mw_1
f2 = p_ref - p_m
f3 = (p_gov_1 - x_gov_1)/T_b_1
f4 = 1/0.1*(p_m_1 - p_m_meas_1) 

g_grid_p = P_t_1*S_b_1/S_b - (v_1*v_0*sin(theta_1 - theta_0))/X_l_1
g_grid_q = Q_t_1*S_b_1/S_b + (v_1*v_0*cos(theta_1 - theta_0))/X_l_1 - v_1**2/X_l_1

NameError: name 'omega_ref' is not defined